In [1]:
import numpy as np
import math
import skimage.io as io
import matplotlib.pyplot as plt
from skimage.transform import rotate
import os
import cv2
import tqdm

In [2]:
def augmentImage(im_a):
    data_train = []

    flip_0 = im_a
    #flip_45 = rotate(im_a,angle=45,mode='wrap')
    flip_90 = rotate(im_a,angle=90,mode='wrap')
    #flip_135 = rotate(im_a,angle=135,mode='wrap')
    flip_180 = rotate(im_a,angle=180,mode='wrap')
    #flip_225 = rotate(im_a,angle=225,mode='wrap')
    flip_270 = rotate(im_a,angle=270,mode='wrap')
    #flip_315 = rotate(im_a,angle=315,mode='wrap')

    data_train.append(flip_0)
    #data_train.append(flip_45)
    data_train.append(flip_90)
    #data_train.append(flip_135)
    data_train.append(flip_180)
    #data_train.append(flip_225)
    data_train.append(flip_270)
    #data_train.append(flip_315)

    data_train.append(np.flipud(flip_0))
    #data_train.append(np.flipud(flip_45))
    data_train.append(np.flipud(flip_90))
    #data_train.append(np.flipud(flip_135))
    data_train.append(np.flipud(flip_180))
    #data_train.append(np.flipud(flip_225))
    data_train.append(np.flipud(flip_270))
    #data_train.append(np.flipud(flip_315))
    
    return data_train

In [3]:
#Para la generación del etiquetado ground truth, utilizaremos la menor cantidad de cuadrados
#que se puedan generar a partir de la imagen 

def mcd(n,m) : # Calcula el maximo común divisor 
  if n == 0:
    return m
  return mcd(m % n, n)

def squares(n, m) : # Devuelve la cantidad de cuadrados que se extraen de la imagen
    return n/mcd(n,m) * m/mcd(n,m) 
    
overlap = 0.1 #porcentaje de overlap entre cuadrados 
def subdivs(image,index,tfile) :
    h = image.shape[0]
    w = image.shape[1]

    maxSqr = squares(h,w)
    sqrFirst = mcd(h,w)
    sqrFinal = sqrFirst + overlap*sqrFirst
    step = sqrFirst - overlap*sqrFirst
    subdivs = []
    
    #Obtenemos todas las subdivisiones de la imagen 
    for i in range(0, int(h)-int(sqrFinal)+1, int(step)):
        subdivs.append([])
        for j in range(0, int(w)-int(sqrFinal)+1, int(step)):
            patch = image[i:i+int(sqrFinal), j:j+int(sqrFinal), :]
            subdivs[-1].append(patch)
    
    #Con las subdivisiones, aumentamos la imagen
    count = 0
    subdivs = np.array(subdivs)
    rangei = subdivs.shape[0]
    rangej = subdivs.shape[1]
    
    for i in tqdm.tqdm(range(rangei)):
        for j in range(rangej):
            
            img = subdivs[i,j,:,:,:]
            augmentedData = augmentImage(img)
            
            for div in augmentedData:
                res = cv2.resize(div, dsize=(512, 512), interpolation=cv2.INTER_NEAREST)
                
                if(tfile=='tiff'):
                    plt.imsave("dataset/test/"+ str(index) + "/" + str(count) + ".tiff", res)
                else:
                    plt.imsave("dataset/test/"+ str(index) + "/" + str(count) + ".jpg", res)
                count+=1

In [4]:
#Directorio de imagenes
col_dir = 'dataset_preprocess/test/'

#Creando una coleccion con todas las imagenes 
images = []
count = 0
for filename in os.listdir(col_dir):
    img = io.imread(col_dir+str(count)+'.JPG')
    imgGT = io.imread(col_dir+str(count)+'.tif')
    
    os.makedirs(os.path.dirname("dataset/test/"+ str(count) + "/"))
    if img is not None:
        subdivs(img,count,'jpg')
        subdivs(imgGT,count,'tiff')
        count+=1
    
    if(count==1):break; #CAMBIAR CUANDO SE HAGAN MAS GT

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:22<00:00, 11.04s/it]
